In [2]:
# magic command to reload modules
%load_ext autoreload
%autoreload 2

## experiement trials analysis

In [3]:
# append sys.path
import sys
sys.path.append('../src')

from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

from simulator.DM_model import DM_model
from parse_data.decode_parameter import decode_mat_fitted_parameters
from parse_data.parse_trial_data import parse_trial_data, compute_subject_acc, correctness_of_array
import os

In [4]:
import matplotlib as mpl
# remove top and right axis from plots
mpl.rcParams["axes.spines.right"] = True
mpl.rcParams["axes.spines.top"] = True

# remove all edges
mpl.rcParams["axes.edgecolor"] = "k"
mpl.rcParams['axes.linewidth'] = 2

font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}
mpl.rc('font', **font)

In [5]:
!pwd

/home/wehe/tmp/NSC/codes/notebook


## 

In [6]:
from dataset.dataset_pipeline import seqC_pattern_summary, process_x_seqC_part
import warnings

config = {
    'train_data':{
        'seqC_process': 'summary',
        'summary_type': 0,
        },
    
}

subj_id = 3
chosen_dur_list = [3, 13]
chosen_MS_list = [0.2, 0.8]

data_path = '../../data'
trial_data_path = os.path.join(data_path, 'trials.mat')
trials = parse_trial_data(trial_data_path)
print(trials.keys())

dict_keys(['pulse', 'dur', 'MS', 'nRight', 'nLeft', 'nPulse', 'hist_nRelse', 'hist_nLelse', 'hist_nRsame', 'hist_nLsame', 'hist_nRoppo', 'hist_nLoppo', 'chooseR', 'subjID', 'correct', 'info'])


In [9]:
subject_idx = trials['subjID']==subj_id
subject_idx = subject_idx.reshape(-1)

seqC_subj = trials['pulse'][subject_idx]
MS_subj = trials['MS'][subject_idx][:,0]
ch_subj = trials['chooseR'][subject_idx][:,0]
dur_subj = trials['dur'][subject_idx][:,0]

idx_chosen = [i for i in range(len(dur_subj)) if MS_subj[i] in chosen_MS_list and dur_subj[i] in chosen_dur_list]
seqC_chosen = seqC_subj[idx_chosen]
ch_chosen = ch_subj[idx_chosen].reshape(-1,1)

x_seqC_chosen = process_x_seqC_part(seqC_chosen, config)
# x_seqC_chosen = seqC_pattern_summary(seqC_chosen, summary_type=0)
x_o_chosen = np.concatenate([x_seqC_chosen, ch_chosen], axis=-1) # DM*S, L_x+1

# raise a warning if x_seqC_chosen.shape[0] != 700*len(chosen_dur_list)*len(chosen_MS_list)
if x_seqC_chosen.shape[0] != 700*len(chosen_dur_list)*len(chosen_MS_list):
    warnings.warn('x_seqC_chosen.shape[0] != 700*len(chosen_dur_list)*len(chosen_MS_list)')



# for i in range(len(dur_subj)):
#     if MS_subj[i] in chosen_MS_list and dur_subj[i] in chosen_dur_list:
#         idx_chosen.append(i)



# seqC_chosen = []
# ch_chosen = []
# for d in chosen_dur_list:
#     dur_idx = np.where(dur_subj==d)[0]
#     seqC_subj_dur = seqC_subj[dur_idx]
#     MS_subj_dur = MS_subj[dur_idx]
#     ch_subj_dur = ch_subj[dur_idx]
    
#     for ms in chosen_MS_list:
#         ms_idx = np.where(MS_subj_dur==ms)[0]
#         seqC_subj_dur_ms = seqC_subj_dur[ms_idx][:chosen_seqC_subset_len, :]
#         ch_subj_dur_ms = ch_subj_dur[ms_idx][:chosen_seqC_subset_len]
        
#     seqC_chosen.append(seqC_subj_dur_ms)
#     ch_chosen.append(ch_subj_dur_ms)

# seqC_chosen_ = np.array(seqC_chosen)
# DM, S, L_x = seqC_chosen_.shape
# seqC_chosen=np.transpose(seqC_chosen_, (2, 0, 1)) # L_x, DM, S
# seqC_chosen = seqC_chosen.reshape(L_x, DM*S)
# seqC_chosen = np.transpose(seqC_chosen, (1, 0)) # DM*S, L_x

# x_seqC = process_x_seqC_part(seqC_chosen, config)
# # x_seqC = seqC_pattern_summary(seqC_chosen, summary_type=0)

# ch_chosen_ = np.array(ch_chosen)
# ch_chosen = ch_chosen_.reshape(-1,1) #

# x_o_chosen = np.concatenate([seqC_chosen, ch_chosen], axis=-1) # DM*S, L_x+1

In [52]:
MS_subj == 0.4

array([ True,  True, False, ...,  True, False,  True])

In [ ]:
seqC_subj_dur = seqC_subj[dur_idx[:,0],:]
MS = trials['MS'][subject_idx][dur_idx]
chosen_MS

# preprocess seqC data
x_seqC = process_x_seqC_part(seqC_subj_dur, config)
# x_seqC = seqC_pattern_summary(seqC_subj, summary_type=0)
# x_seqC = seqC_pattern_summary(seqC_subj, summary_type=1)

# merge seqC and chooseR
x_o = np.concatenate((x_seqC, ch_subj), axis=1)

In [153]:
def get_xo(
    trial_data_path='../..data/trials.mat',
    subj_id = 2,
    chosen_dur = 11, # 0 means all durations
    ): # data observations

    
    trials = parse_trial_data(trial_data_path)

    subject_idx = trials['subjID']==subj_id
    subject_idx = subject_idx.reshape(-1)
    seqC_subj = trials['pulse'][subject_idx]
    ch_subj = trials['chooseR'][subject_idx]

    dur = trials['dur'][subject_idx]
    dur_idx = dur==chosen_dur
    if chosen_dur == 0: # choose all durations
        seqC_subj_dur = seqC_subj 
    else:
        seqC_subj_dur = seqC_subj[dur_idx[:,0],:]

    # preprocess seqC data
    x_seqC = process_x_seqC_part(seqC_subj_dur, config)
    # x_seqC = seqC_pattern_summary(seqC_subj, summary_type=0)
    # x_seqC = seqC_pattern_summary(seqC_subj, summary_type=1)

    # merge seqC and chooseR
    x_o = np.concatenate((x_seqC, ch_subj), axis=1)
    return x_o

config = {
    
}
get_xo(config, trial_data_path, subj_id = 2, chosen_dur = 11)

In [151]:
x_o.shape

(14700, 9)

In [12]:
import h5py
import torch
f = h5py.File('../../data/datasets/test_train_data.h5', 'r')
theta = f['theta'][:]
x = f['x'][:]
f.close()
print('x.shape', x.shape, '\ttheta.shape', theta.shape)

x.shape (2, 2, 2, 3, 5, 16) 	theta.shape (2, 2, 2, 3, 5, 4)


In [13]:
D,M,S,T,C,L_x = x.shape
# 0,1,2,3,4,5
_,_,_,_,_,L_theta = theta.shape
print(' D M S T C Lx Ltheta\n', D,M,S,T,C,L_x, L_theta)
x = torch.from_numpy(x)
theta = torch.from_numpy(theta)

x_ = x.permute(0,3,4,5,1,2).reshape(D,T,C,L_x,M*S) # D,T,C,L_x,M*S
x_ = x_.permute(0,1,2,4,3)  # D,T,C,M*S,L_x
                            # 0,1,2,3  ,4

theta_ = theta.permute(0,3,4,5,1,2).reshape(D,T,C,L_theta,M*S) # D,T,C,L_theta,M*S
theta_ = theta_.permute(0,1,2,4,3)  # D,T,C,M*S,L_theta
                                    # 0,1,2,3  ,4

# randomize the order of the same duration D
x_processed = torch.empty_like(x_)
theta_processed = torch.empty_like(theta_)
for d in range(D):
    for t in range(T):
        # for c in range(C):
        x_temp = x_[d,t,:,:,:]
        theta_temp = theta_[d,t,:,:,:]
        
        idx = torch.randperm(M*S)
        x_processed[d,t,:,:,:] = x_temp[:,idx,:] # M*S,L_x
        theta_processed[d,t,:,:,:] = theta_temp[:,idx,:] # M*S,L_theta

# further reshape x_processed and theta_processed
x_ = x_processed
# to shape T,C,L_x,D,M*S
x_ = x_.permute(1,2,4,0,3).reshape(T,C,L_x,D*M*S)# T,C,L_x,D*M*S
                                                    # 0,1,2  ,3
# to shape L_x, D*M*S, T, C
x_ = x_.permute(2, 3, 0, 1).reshape(L_x, D*M*S, T*C) # L_x, D*M*S, T*C
                                                    # 0  ,1     ,2
x_ = x_.permute(2,1,0) # T*C, D*M*S, L_x
# to shape T, C, D*M*S, L_x
# x_ = x_.permute(0,1,3,2)

theta_ = theta_processed
# to shape T,C,L_theta,D,M*S
theta_ = theta_.permute(1,2,4,0,3).reshape(T,C,L_theta,D*M*S) # T,C,L_theta,D*M*S
                                                                # 0,1,2      ,3
# to shape L_theta, D*M*S, T, C
theta_ = theta_.permute(2, 3, 0, 1).reshape(L_theta, D*M*S, T*C) # L_theta, D*M*S, T*C
                                                                # 0      ,1     ,2 
theta_ = theta_.permute(2,1,0) # T*C, D*M*S, L_theta
# to shape T, C, D*M*S, L_theta
# theta_ = theta_.permute(0,1,3,2)

# output shape: T*C, 0, L_theta
theta_ = theta_[:,0,:]
print('x.shape', x_.shape, 'theta.shape', theta_.shape)

 D M S T C Lx Ltheta
 2 2 2 3 5 16 4
x.shape torch.Size([15, 8, 16]) theta.shape torch.Size([15, 4])


In [143]:
*a.shape

SyntaxError: can't use starred expression here (1871072887.py, line 4)